# Math Scores

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sn
import matplotlib.pyplot as plt
os.chdir('C:\\Users\\mhous\\Test Scores')

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

train_mat = pd.read_csv('train_mat.csv')
test_mat = pd.read_csv('test_mat.csv')

## Regression

We begin with regression on the G3 scores, which range from 1-20. We create a baseline model, which is the mean G3 score in training data. We can then compare the RMSE of this model to the random forest models. We then train a baseline random forest model, and then use RandomizedSearchCV to tune the hyperparameters. Finally we evaluate the best estimator found in the RandomizedSearchCV and analyze the feature importances.

In [2]:
baseline = train_mat['G3'].mean()
baseline_array = np.full(len(test_mat), baseline)
rmse = np.sqrt(mean_squared_error(test_mat['G3'], baseline_array))
print("RMSE of baseline model: %.3f" %rmse)

RMSE of baseline model: 4.912


In [3]:
X_train = train_mat.drop(['G1', 'G2', 'G3', 'G3_five_levels', 'G3_pass_fail'], axis=1)
y_train = train_mat['G3']

X_test = test_mat.drop(['G1', 'G2', 'G3', 'G3_five_levels', 'G3_pass_fail'], axis=1)
y_test = test_mat['G3']

In [4]:
def evaluate_regression_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    r2 = r2_score(y_test, predictions)
    print("RMSE: % .3f" %rmse)
    print("R-Squared: % .3f"%r2)

In [5]:
rf = RandomForestRegressor(random_state=1)
rf.fit(X_train, y_train)
evaluate_regression_model(rf, X_test, y_test)

RMSE:  3.899
R-Squared:  0.370


In [6]:
n_estimators = [int(x) for x in np.linspace(100,  2000, 20)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2,5,10]
min_samples_leaf = [1,2,4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
rf_random = RandomizedSearchCV(RandomForestRegressor(random_state=1), param_distributions = random_grid, n_iter = 50, cv = 5, verbose=1, random_state=1, n_jobs=-1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  3.2min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100, 1200,
                                                         1300, 1400, 1500, 1600,
                                                         1700, 1800, 

In [8]:
print(rf_random.best_params_)
rf_best = rf_random.best_estimator_

{'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': None, 'bootstrap': True}


In [9]:
evaluate_regression_model(rf_best, X_test, y_test)

RMSE:  3.886
R-Squared:  0.374


The RandomizedSearch improved our model very slightly, with an 0.013 decrease in RMSE and a .004 improvement in R-Squared. Let's now look at the most important features.

In [10]:
def variable_importances(model):
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]
    print("Feature ranking:")
    for f in range(10):
        print("%d. Feature: %s (%f)" % (f + 1, X_train.columns[indices[f]], importances[indices[f]]))

In [11]:
variable_importances(rf_best)

Feature ranking:
1. Feature: absences (0.221063)
2. Feature: failures (0.155510)
3. Feature: goout (0.041093)
4. Feature: health (0.037930)
5. Feature: studytime (0.035995)
6. Feature: freetime (0.035763)
7. Feature: Medu (0.033888)
8. Feature: traveltime (0.033178)
9. Feature: age (0.031372)
10. Feature: schoolsup (0.025548)


As one might expect, the number of absences and the number of failures are the most important variables when it comes to the regression model. There is a sharp drop off in importance with the rest of the features that account for 4% or less. 

Overall, our random forest regression model is an improvement over the baseline model, but since the RMSE being ~3.9 means we aren't very accurate when it comes to predicting final test scores. This might be due to an insufficient amount of data, missing important features such as income, or just the irreducible error in predicting final test scores.

## Five Levels

Now we move onto a five level classification task. The five levels classification is as follows:

| Level      | G3 Score |
| :----------- | -----------|
| 1      |    16-20   |
| 2   |    14-15     |
| 3   |   12-13     |
| 4   |   10-11      |
| 5   | 0-9 |

In this case, our baseline model is the most common class in the training data. 

We again begin with creating a base model, then using RandomizedSearchCV to tune the hyperparameters, then evaluating the model using the classification report and confusion matrix. 



In [12]:
X_train = train_mat.drop(['G1', 'G2', 'G3', 'G3_five_levels', 'G3_pass_fail'], axis=1)
y_train = train_mat['G3_five_levels']

X_test = test_mat.drop(['G1', 'G2', 'G3', 'G3_five_levels', 'G3_pass_fail'], axis=1)
y_test = test_mat['G3_five_levels']

In [13]:
most_common_class = train_mat['G3_five_levels'].value_counts().sort_values(ascending=False).index[0]
classified_correctly = 0
for i in range(len(test_mat)):
    if test_mat['G3_five_levels'][i] == most_common_class:
        classified_correctly +=1 
print("Mean accuracy of baseline model: " + str(round(classified_correctly/len(test_mat), 3)))

Mean accuracy of baseline model: 0.35


In [14]:
def evaluate_classification_model(model, X_test, y_test):
    model_base_score = rf.score(X_test, y_test)
    print("Mean accuracy: % .3f" % model_base_score)

In [15]:
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)
evaluate_classification_model(rf, X_test, y_test)

Mean accuracy:  0.317


In [16]:
rf_random = RandomizedSearchCV(RandomForestClassifier(random_state=1), param_distributions = random_grid, n_iter = 50, cv = 5, verbose=1, random_state=1, n_jobs=-1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.4min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100, 1200,
                                                         1300, 1400, 1500, 1600,
                                                         1700, 1800,

In [17]:
print(rf_random.best_params_)
rf_best = rf_random.best_estimator_

{'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


In [18]:
print("Mean accuracy: " + str(round(rf_best.score(X_test, y_test), 4)))

Mean accuracy: 0.35


In [19]:
predictions = rf_best.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       0.33      0.14      0.20         7
           2       0.12      0.10      0.11        10
           3       0.00      0.00      0.00         8
           4       0.26      0.36      0.30        14
           5       0.52      0.67      0.58        21

    accuracy                           0.35        60
   macro avg       0.25      0.25      0.24        60
weighted avg       0.30      0.35      0.32        60



In [20]:
confusion_matrix(y_test, predictions)

array([[ 1,  2,  0,  3,  1],
       [ 1,  1,  1,  4,  3],
       [ 0,  1,  0,  4,  3],
       [ 1,  2,  0,  5,  6],
       [ 0,  2,  2,  3, 14]], dtype=int64)

We find that our five level classification has a mean accuracy of .35 and that it is much better at predicting who will do worse than who will do well. For example, the model predicted that 3 people would score a 1, the highest possible level, but only one of those predictions was true (precision = .33). Additionally, there were 7 people who scored a 1, and only 1 of those 7 was correctly predicted (recall = .14). The model performs better at predicting who is likely to fail, where the precision and recall values are .52 and .67, respectively. 

Ultimately though, our five level classification model was equally accurate as just predicting every student will score a 5, so the model really does not do a good job of predicting how students score on the five level scale. 

In [21]:
variable_importances(rf_best)

Feature ranking:
1. Feature: absences (0.084653)
2. Feature: failures (0.070682)
3. Feature: Medu (0.048718)
4. Feature: goout (0.046857)
5. Feature: health (0.044957)
6. Feature: freetime (0.043805)
7. Feature: Walc (0.042381)
8. Feature: studytime (0.040545)
9. Feature: famrel (0.036357)
10. Feature: age (0.035003)


The variable importances in this model are much more evenly distributed. Absences again is the most predictive variable, but interestingly failures in now the 6th most important variable.

## Pass/Fail

Finally, we move onto our pass/fail classification system. Passing counts as scoring a 10 or higher on the G3 grades, and failing is scoring 0-9. Our baseline is again the most common class in the training data. 

In [22]:
most_common_class = train_mat['G3_pass_fail'].value_counts().sort_values(ascending=False).index[0]
classified_correctly = 0
for i in range(len(test_mat)):
    if test_mat['G3_pass_fail'][i] == most_common_class:
        classified_correctly +=1 
print("Mean accuracy of baseline model: " + str(round(classified_correctly/len(test_mat), 3)))

Mean accuracy of baseline model: 0.65


In [23]:
X_train = train_mat.drop(['G1', 'G2', 'G3', 'G3_five_levels', 'G3_pass_fail'], axis=1)
y_train = train_mat['G3_pass_fail']

X_test = test_mat.drop(['G1', 'G2', 'G3', 'G3_five_levels', 'G3_pass_fail'], axis=1)
y_test = test_mat['G3_pass_fail']

In [24]:
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train)
rf_base_score = round(rf.score(X_test, y_test), 3)
print("Mean accuracy: " + str(rf_base_score))

Mean accuracy: 0.717


In [25]:
rf_random = RandomizedSearchCV(RandomForestClassifier(random_state=1), param_distributions = random_grid, n_iter = 50, cv = 5, verbose=1, random_state=1, n_jobs=-1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.7min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100, 1200,
                                                         1300, 1400, 1500, 1600,
                                                         1700, 1800,

In [26]:
print(rf_random.best_params_)
rf_best = rf_random.best_estimator_

{'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': None, 'bootstrap': False}


In [27]:
print("Mean accuracy: " + str(round(rf_best.score(X_test, y_test), 3)))

Mean accuracy: 0.7


In [28]:
predictions = rf_best.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.67      0.29      0.40        21
           1       0.71      0.92      0.80        39

    accuracy                           0.70        60
   macro avg       0.69      0.60      0.60        60
weighted avg       0.69      0.70      0.66        60



In [29]:
confusion_matrix(y_test, predictions)

array([[ 6, 15],
       [ 3, 36]], dtype=int64)

Interestingly, this model is better at predicting who will pass than who will fail. The model correctly predicted 36/39 students that actually passed, but also predicted that 15 students would pass that actually failed. This goes in hand with the low recall in predicting who would fail, correctly predictingg only 6/21. 

Our pass/fail random forest model performs better than our baseline model does by 5%, which shows the model does a better job at predicting  who will pass/fail than by random chance.

In [30]:
variable_importances(rf_best)

Feature ranking:
1. Feature: failures (0.152744)
2. Feature: absences (0.104087)
3. Feature: goout (0.068500)
4. Feature: age (0.046360)
5. Feature: Medu (0.041301)
6. Feature: Walc (0.036933)
7. Feature: Fedu (0.036532)
8. Feature: health (0.034572)
9. Feature: freetime (0.030661)
10. Feature: famrel (0.029877)


Again, in the top 10 most important features we see a lot of similarities as before. Failures makes its way to the most important feature, and absences is again one of the most explanatory features>